In [1]:
using Pkg
using JuMP
using Gurobi
#using GLPK
using XLSX
using DataFrames

In [2]:
using LinearAlgebra

In [3]:
shared_cand = DataFrame(XLSX.readtable("CandLine.xlsx", "Taul1", header=true)...) #Dataframe of shared candidate lines
int_cand =  DataFrame(XLSX.readtable("CandLineInt.xlsx", "Taul1", header=true)...) #Dataframe of internal candidate lines
shared_ex =  DataFrame(XLSX.readtable("SharedEline.xlsx", "Taul1", header=true)...) #Dataframe of shared existing lines
int_ex = DataFrame(XLSX.readtable("Tran.xlsx", "Taul1", header=true)...) #Dataframe of internal existing lines
gen =  DataFrame(XLSX.readtable("Gen.xlsx" , "Taul1", header=true)...) #Dataframe of generators
load =  DataFrame(XLSX.readtable("Load.xlsx", "Taul1", header=true)...) #Dataframe of loads
##An example of the load dataframe can be found as follows

,zoneNum,lNodeID,P_load1,P_load2,P_load3,P_load4
,Any,Any,Any,Any,Any,Any
1,1,2,-21.7,-10.85,-16.275,-5.425
2,1,3,-94.2,-47.1,-70.65,-23.55
3,1,4,-47.8,-23.9,-35.85,-11.95
4,1,5,-7.6,-3.8,-5.7,-1.9
5,1,6,-11.2,-5.6,-8.4,-2.8
6,1,9,-29.5,-14.75,-22.125,-7.375
7,1,10,-9,-4.5,-6.75,-2.25
8,1,11,-3.5,-1.75,-2.625,-0.875
9,1,12,-6.1,-3.05,-4.575,-1.525


In [4]:
cand_from(i) = (shared_cand.nodeZone1 .== i)
cand_to(i) = (shared_cand.nodeZone2 .== i)
ex_from(i) = (shared_ex.nodeZone1 .== i)
ex_to(i) = (shared_ex.nodeZone2 .== i)

ex_to (generic function with 1 method)

In [5]:
l(i) = load[load.zoneNum .== i, :] # load within zone i
g(i) = gen[gen.zoneNum .== i, :]   # generators within zone i
shared_c(i) = vcat(shared_cand[shared_cand.nodeZone1 .== i,:] , shared_cand[shared_cand.nodeZone2 .== i, :]) #shared candidate lines within zone i
int_c(i) = int_cand[int_cand.zoneNum .== i, :]   # number of internal candidate lines within zone i
shared_e(i) = vcat(shared_ex[shared_ex.nodeZone1 .== i,:] , shared_ex[shared_ex.nodeZone2 .== i, :]) #shared existing lines within zone i
int_e(i) =int_ex[int_ex.zoneNum .== i, :]       # internal existing lines within zone i
MC(i) = (g(i).C2 .* (g(i).PgMax .^ 2) .+ g(i).C1 .* g(i).PgMax .- g(i).C2 .*(g(i).PgMin .^ 2) .- g(i).C1 .* g(i).PgMin) ./ (g(i).PgMax .- g(i).PgMin) #Marginal cost of generators within zone i
bin_c(i) = (shared_cand.nodeZone1 .== i) + (shared_cand.nodeZone2 .== i) # A binary vector through which we can check if the shared candidate lines belong to zone i
bin_e(i) = (shared_ex.nodeZone1 .== i) + (shared_ex.nodeZone2 .== i) # A binary vector through which we can check if the shared existing lines belong to zone i

bin_e (generic function with 1 method)

In [6]:
m(i) = setdiff([1,2,3], [i])

m (generic function with 1 method)

In [7]:
function zonal_app(i, N, cand_lambda, ex_lambda,shared_line_decision_var, int_line_decision_var,
        shared_cand_angle_self, shared_ex_angle_self, shared_cand_angle_1, shared_ex_angle_1, 
        shared_cand_angle_2, shared_ex_angle_2)
    
    Mod4 = Model(Gurobi.Optimizer)
    
    
    @variable(Mod4,0 <= gen_var[1:N]) # generation at each node
    @variable(Mod4,shared_cand_flow[1:nrow(shared_cand)])  #Power flowing into shared candidate lines 
    @variable(Mod4,int_cand_flow[1:nrow(int_c(i))]) #Power flowing into internal candidate lines 
    @variable(Mod4,0 <= shared_cand_angle[1:nrow(shared_cand),1:2]<= 2*pi) #Phase angle decision for shared candidate lines
    @variable(Mod4,0 <= int_cand_angle[1:nrow(int_c(i)),1:2]<= 2*pi) #Phase angle decision for internal candidate lines
    @variable(Mod4,shared_ex_flow[1:nrow(shared_ex)])  #Power flowing on shared existing shared lines
    @variable(Mod4,int_ex_flow[1:nrow(int_e(i))])  #Power flowing on internal existing lines
    @variable(Mod4,0 <= shared_ex_angle[1:nrow(shared_ex),1:2]<= 2*pi) #Phase angle for existing shared lines
    @variable(Mod4,0 <= int_ex_angle[1:nrow(int_e(i)),1:2]<= 2*pi) #Phase angle for internal existing lines
    
    omega = 0.36;
    eta = 0.6;
    
    @expression(Mod4, total_cost, sum((gen_var[n] .* sum((g(i).gNodeID .== n) .* MC(i))) for n in 1:N)
        .+ sum(sum(eta * shared_ex_angle[h,1] .* (shared_ex_angle_self[h,1] .- shared_ex_angle_1[h,1]) .+
            eta * shared_ex_angle[h,1] .* (shared_ex_angle_self[h,1] .- shared_ex_angle_2[h,1]) .+
            eta * shared_ex_angle[h,2] .* (shared_ex_angle_self[h,2] .- shared_ex_angle_1[h,2]) .+
            eta * shared_ex_angle[h,2] .* (shared_ex_angle_self[h,2] .- shared_ex_angle_2[h,2]) .+
            shared_ex_angle[h,1] .* ex_lambda[h,1] .+ 
            shared_ex_angle[h,2] .* ex_lambda[h,2] for h in 1:nrow(shared_ex))) .+
            sum(sum((omega /2) * (shared_ex_angle[:,1] .- shared_ex_angle_self[:,1]) .^ 2)) .+
            sum(sum((omega /2) * (shared_ex_angle[:,2] .- shared_ex_angle_self[:,2]) .^ 2))
        .+ sum(sum(eta * shared_cand_angle[c,1] .* (shared_cand_angle_self[c,1] .- shared_cand_angle_1[c,1]) .+
            eta * shared_cand_angle[c,1] .* (shared_cand_angle_self[c,1] .- shared_cand_angle_2[c,1]) .+
            eta * shared_cand_angle[c,2] .* (shared_cand_angle_self[c,2] .- shared_cand_angle_1[c,2]) .+
            eta * shared_cand_angle[c,2] .* (shared_cand_angle_self[c,2] .- shared_cand_angle_2[c,2]) .+
            shared_cand_angle[c,1] .* cand_lambda[c,1] .+ 
            shared_cand_angle[c,2] .* cand_lambda[c,2] for c in 1:nrow(shared_cand))) .+
            sum(sum((omega /2) * (shared_cand_angle[:,1] .- shared_cand_angle_self[:,1]) .^ 2)) .+
            sum(sum((omega /2) * (shared_cand_angle[:,2] .- shared_cand_angle_self[:,2]) .^ 2)))
            
            
            
            
    for n in 1:N
        # Power balance constraint for each node
        @constraint(Mod4, sum(g(i).gNodeID .== n) .* gen_var[n] .+ sum(l(i).P_load1 .* (l(i).lNodeID .== n)) .==
            sum((shared_cand.tNodeID1 .== n) .*bin_c(i) .* shared_cand_flow) .- sum((shared_cand.tNodeID2 .== n) .* bin_c(i) .* shared_cand_flow) .+
            sum((shared_ex.tNodeID1 .== n) .* bin_e(i) .* shared_ex_flow) .- sum((shared_ex.tNodeID2 .== n) .* bin_e(i) .* shared_ex_flow) .+
            sum((int_c(i).tNodeID1 .== n) .* int_cand_flow) .- sum((int_c(i).tNodeID2 .== n) .* int_cand_flow) .+
            sum((int_e(i).tNodeID1 .== n) .* int_ex_flow) .- sum((int_e(i).tNodeID2 .== n) .* int_ex_flow))
        #Lower limit for generation of each node
        @constraint(Mod4, sum(g(i).gNodeID .== n) .* gen_var[n] .<= sum((g(i).gNodeID .== n) .* g(i).PgMax))
        #Upper limit for generation of each node
        @constraint(Mod4, sum((g(i).gNodeID .== n) .* g(i).PgMin) .<= sum(g(i).gNodeID .== n) .* gen_var[n])
    end
    
    M=1000 #Big number
    
    #Constraints elated to the candidate shared lines:
    for c in 1:nrow(shared_cand) 
        #Associating the power flowing within the line with the phase angles of nodes:
        @constraint(Mod4,-M .* (1 .- bin_c(i) .* shared_line_decision_var[c]) .<= bin_c(i) .* shared_cand_flow[c] .- ((1 ./ shared_cand.reacT[c]) .* (bin_c(i) .* shared_cand_angle[c,1] .- bin_c(i) .* shared_cand_angle[c,2])))
        
        @constraint(Mod4, bin_c(i) .* shared_cand_flow[c] .- ((1 ./ shared_cand.reacT[c]) .* (bin_c(i) .* shared_cand_angle[c,1] .- bin_c(i) .* shared_cand_angle[c,2])) .<= M .* (1 .- bin_c(i) .* shared_line_decision_var[c]))
        #limiting the upper bound of power flow flowing within candidate shared lines
        @constraint(Mod4, bin_c(i) .* shared_cand_flow[c] .<= bin_c(i) .* shared_line_decision_var[c] .* shared_cand.ptMax[c])
        #Limiting the lower bound of power flowing within the candidate shared lines
        @constraint(Mod4, -shared_cand.ptMax[c] .* bin_c(i) .* shared_line_decision_var[c] .<= bin_c(i) .* shared_cand_flow[c])
        @constraint(Mod4, shared_cand_angle[5,2] .== 0)
    end
    
    #The same constraints related to the candidate internal lines:
    for c in 1:nrow(int_c(i))
        @constraint(Mod4,-M .* (1 .- int_line_decision_var[c]) .<= int_cand_flow[c] .- ((1 ./ int_c(i).reacT[c]) .* (int_cand_angle[c,1] .- int_cand_angle[c,2])))
        @constraint(Mod4, int_cand_flow[c] .- ((1 ./ int_c(i).reacT[c]) .* (int_cand_angle[c,1] .- int_cand_angle[c,2])) .<= M .* (1 .- int_line_decision_var[c]))
        @constraint(Mod4, int_cand_flow[c] .<= int_line_decision_var[c] .* int_c(i).ptMax[c])
        @constraint(Mod4, -int_c(i).ptMax[c] .*int_line_decision_var[c] .<= int_cand_flow[c])
    end
    
    #The same constraints related to the shared existing lines:
    for h in 1:nrow(shared_ex)
        @constraint(Mod4, bin_e(i) .* shared_ex_flow[h] .== (1 ./ shared_ex.reacT[h]) .* (bin_e(i) .* shared_ex_angle[h,1] .- bin_e(i) .* shared_ex_angle[h,2]))
        @constraint(Mod4, bin_e(i) .* shared_ex_flow[h] .<= bin_e(i) .* shared_ex. ptMax[h])
        @constraint(Mod4, bin_e(i) .* -shared_ex.ptMax[h] .<= bin_e(i) .* shared_ex_flow[h])
        @constraint(Mod4, shared_ex_angle[1,2] .== 0)
        @constraint(Mod4, shared_ex_angle[7,2] .== 0)

    end
    #The same constraints related to the internal existing lines:
    
    for h in 1:nrow(int_e(i))
        @constraint(Mod4, int_ex_flow[h] .== (1 ./ int_e(i).reacT[h]) .* (int_ex_angle[h,1] .- int_ex_angle[h,2]))
        @constraint(Mod4, int_ex_flow[h] .<= int_e(i).ptMax[h])
        @constraint(Mod4, -int_e(i).ptMax[h] .<= int_ex_flow[h])
    end
    
    #The model objective is to minimize the total cost:
    
    @objective(Mod4, Min, total_cost)
    optimize!(Mod4)
    if termination_status(Mod4) == MOI.OPTIMAL #If the solution is optimal then it returns the variables
    
        shared_cand_power = value.(shared_cand_flow)
        shared_cand_phase_angle = value.(shared_cand_angle)
        shared_ex_power = value.(shared_ex_flow)
        shared_ex_phase_angle = value.(shared_ex_angle)
        gen_power = value.(gen_var)
        int_cand_power = value.(int_cand_flow)
        int_cand_phase_angle = value.(int_cand_angle)
        obj_value = objective_value(Mod4)
        generation = value.(gen_var)
        [obj_value, shared_cand_power,shared_cand_phase_angle,shared_ex_power,shared_ex_phase_angle,gen_power, int_cand_power, int_cand_phase_angle, generation]

    end
end
    
    

zonal_app (generic function with 1 method)

In [8]:
f=ones(12000)
h=0
cand_lambda_1= zeros(12000,nrow(shared_cand),2)
ex_lambda_1 = zeros(12000,nrow(shared_ex),2)
cand_lambda_2= zeros(12000,nrow(shared_cand),2)
ex_lambda_2 = zeros(12000,nrow(shared_ex),2)
cand_lambda_3= zeros(12000,nrow(shared_cand),2)
ex_lambda_3 = zeros(12000,nrow(shared_ex),2)
cand_angle_1 = zeros(12000,nrow(shared_cand),2)
c_angle_1 = zeros(12000,nrow(shared_cand),2)
cand_angle_2 = zeros(12000,nrow(shared_cand),2)
c_angle_2 = zeros(12000,nrow(shared_cand),2)
cand_angle_3 = zeros(12000,nrow(shared_cand),2)
c_angle_3 = zeros(12000,nrow(shared_cand),2)

ex_angle_1 = zeros(12000,nrow(shared_ex),2)
e_angle_1 = zeros(12000,nrow(shared_ex),2)
ex_angle_2 = zeros(12000,nrow(shared_ex),2)
e_angle_2 = zeros(12000,nrow(shared_ex),2)
ex_angle_3 = zeros(12000,nrow(shared_ex),2)
e_angle_3 = zeros(12000,nrow(shared_ex),2)

cand_flow_1 =zeros(12000,nrow(shared_cand))
cand_flow_2 =zeros(12000,nrow(shared_cand))
cand_flow_3 =zeros(12000,nrow(shared_cand))
ex_flow_1 = zeros(12000,nrow(shared_ex))
ex_flow_2 = zeros(12000,nrow(shared_ex))
ex_flow_3 = zeros(12000,nrow(shared_ex))

gen_1 = zeros(12000,nrow(g(1)))
gen_2 = zeros(12000,nrow(g(2)))
gen_3 = zeros(12000,nrow(g(3)))



shared_line_decision_var_m =  
[1.0
 0.0
 0.0
 1.0
 1.0
 0.0]
int_line_decision_var_1 =
[0.0
 1.0
 1.0
 0.0
 1.0
 1.0
 0.0
 1.0
 0.0
 1.0]
int_line_decision_var_2 =
[1.0
 0.0
 0.0
 0.0
 0.0
 1.0
 1.0
 0.0
 0.0
 1.0
 1.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0]
int_line_decision_var_3 =
[0.0
 0.0]

2-element Vector{Float64}:
 0.0
 0.0

In [9]:
    j = 2
    for j in 2:200    # It considers 1000 iteration
        if abs(f[j-1]) > 0.00000001  
         
            R_1 = zonal_app(1, 14, cand_lambda_1[j,:,:], ex_lambda_1[j,:,:],shared_line_decision_var_m, int_line_decision_var_1,
            cand_angle_1[j-1,:,:], ex_angle_1[j-1,:,:], cand_angle_2[j-1,:,:],  ex_angle_2[j-1,:,:], 
            cand_angle_3[j-1,:,:], ex_angle_3[j-1,:,:])
        
            R_2 = zonal_app(2, 30, cand_lambda_2[j,:,:], ex_lambda_2[j,:,:],shared_line_decision_var_m, int_line_decision_var_2,
            cand_angle_2[j-1,:,:], ex_angle_2[j-1,:,:], cand_angle_1[j-1,:,:], ex_angle_1[j-1,:,:], 
            cand_angle_3[j-1,:,:], ex_angle_3[j-1,:,:])       
        
            R_3 = zonal_app(3, 5, cand_lambda_3[j,:,:], ex_lambda_3[j,:,:],shared_line_decision_var_m, int_line_decision_var_3,
            cand_angle_3[j-1,:,:], ex_angle_3[j-1,:,:], cand_angle_1[j-1,:,:],  ex_angle_1[j-1,:,:], 
            cand_angle_2[j-1,:,:], ex_angle_2[j-1,:,:])
        
        
            cand_lambda_1[j+1,:,:] .= cand_lambda_1[j,:,:] .+ 0.1 * (R_1[3] .- (R_2[3] .+ R_3[3]) ./2)
            cand_lambda_2[j+1,:,:] .= cand_lambda_2[j,:,:] .+ 0.1 * (R_2[3] .- (R_1[3] .+ R_3[3]) ./2)
            cand_lambda_3[j+1,:,:] .= cand_lambda_3[j,:,:] .+ 0.1 * (R_3[3] .- (R_1[3] .+ R_2[3]) ./2)
        
        
    
            ex_lambda_1[j+1,:,:] .= ex_lambda_1[j,:,:] .+ 0.1 * (R_1[5] .- (R_2[5] .+ R_3[5]) ./2)
            ex_lambda_2[j+1,:,:] .= ex_lambda_2[j,:,:] .+ 0.1 * (R_2[5] .- (R_1[5] .+ R_3[5]) ./2)
            ex_lambda_3[j+1,:,:] .= ex_lambda_3[j,:,:] .+ 0.1 * (R_3[5] .- (R_1[5] .+ R_2[5]) ./2)
        
            f[j] = sum(sum((R_1[3] .- (R_2[3]./2 .+ R_3[3]./2)).^ 2)) +
                    sum(sum((R_2[3] .- (R_1[3]./2 .+ R_3[3]./2)).^ 2)) +
                    sum(sum((R_3[3] .- (R_1[3]./2 .+ R_2[3]./2)).^ 2)) +
                    sum(sum((R_1[5] .- (R_2[5]./2 .+ R_3[5]./2)).^ 2)) +
                    sum(sum((R_2[5] .- (R_1[5]./2 .+ R_3[5]./2)).^ 2)) +
                    sum(sum((R_3[5] .- (R_2[5]./2 .+ R_1[5]./2)).^ 2))
        
            c_angle_1 = R_1[3]
            c_angle_2 = R_2[3]
            c_angle_3 = R_3[3]
        
            e_angle_1 = R_1[5]
            e_angle_2 = R_2[5]
            e_angle_3 = R_3[5]
            
            cand_flow_1 = R_1[2]
            cand_flow_2 = R_2[2]
            cand_flow_3 = R_3[2]
        
            ex_flow_1 = R_1[4]
            ex_flow_2 = R_2[4]
            ex_flow_3 = R_3[4]
        
            gen_1 = R_1[9]
            gen_2 = R_2[9]
            gen_3 = R_3[9]
            
            
            h= R_2[1] + R_1[1] + R_3[1]
            
            if isdefined(Main, :IJulia)
                Main.IJulia.stdio_bytes[] = 0
            end
        
            print('*') 
        
        else
            print(h)
            print(c_angle_1)
            print(c_angle_2)
            print(c_angle_3)
            print(e_angle_1)
            print(e_angle_2)
            print(e_angle_3)
        end
    end
    

Academic license - for non-commercial use only - expires 2023-06-28
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 500 rows, 146 columns and 674 nonzeros
Model fingerprint: 0x7e8acae9
Model has 28 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [3e+01, 6e+01]
  QObjective range [4e-01, 4e-01]
  Bounds range     [6e+00, 6e+00]
  RHS range        [4e+00, 1e+03]
Presolve removed 486 rows and 110 columns
Presolve time: 0.00s
Presolved: 14 rows, 36 columns, 65 nonzeros
Presolved model has 9 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 2.800e+01
 Factor NZ  : 1.050e+02
 Factor Ops : 1.015e+03 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   2.35447483e+06 -2.09222

In [10]:
h #OF of the second stage

5384.955568547177

In [11]:
c_angle_1

6×2 Matrix{Float64}:
 1.25328  4.16093e-10
 0.0      0.0
 0.0      0.0
 5.75     9.05131e-11
 4.74471  0.0
 0.0      0.0

In [12]:
c_angle_2

6×2 Matrix{Float64}:
 1.25139  8.15516e-7
 0.0      0.0
 0.0      0.0
 5.74999  3.42028e-5
 4.74471  0.0
 0.0      0.0

In [13]:
c_angle_3

6×2 Matrix{Float64}:
 1.25139  1.01061e-5
 0.0      0.0
 0.0      0.0
 5.74999  3.59671e-5
 4.74471  0.0
 0.0      0.0

In [14]:
e_angle_3 

8×2 Matrix{Float64}:
 6.28318      0.0
 0.0          0.0
 0.0          0.0
 0.000553175  0.00056674
 5.99         3.81195e-11
 3.91913      5.82766e-11
 0.000229198  0.0
 6.58079e-11  3.47014

In [15]:
e_angle_2

8×2 Matrix{Float64}:
 6.28317      0.0
 0.0          0.0
 0.0          0.0
 0.000553253  0.00056682
 5.98771      0.00230011
 3.91913      1.17529e-7
 0.000229274  0.0
 1.4884e-5    3.47014

In [16]:
e_angle_1

8×2 Matrix{Float64}:
 6.28319      0.0
 0.0          0.0
 0.0          0.0
 0.000553612  0.00056719
 5.98584      0.00417089
 3.91913      5.84195e-8
 0.000229243  0.0
 7.69922e-6   3.47014

In [17]:
cand_flow_1

6-element Vector{Float64}:
 21.180947594998656
  0.0
  0.0
 99.9999999999834
 28.72099575378764
  0.0

In [18]:
cand_flow_2

6-element Vector{Float64}:
 21.149051955657256
  0.0
  0.0
 99.99917360080232
 28.720995753781274
  0.0

In [19]:
cand_flow_3

6-element Vector{Float64}:
 21.148894941083128
  0.0
  0.0
 99.9991247040594
 28.720995753794337
  0.0

In [20]:
ex_flow_1

8-element Vector{Float64}:
  23.27105669324042
   0.0
   0.0
  -2.4420213512965595e-5
  99.86094994118469
  19.59564580684882
   0.0020840266621603193
 -16.68332113302352

In [21]:
ex_flow_2

8-element Vector{Float64}:
  23.270995840782327
   0.0
   0.0
  -2.439994618433708e-5
  99.92329861873816
  19.5956455112966
   0.0020843064816893917
 -16.683286590653093

In [22]:
ex_flow_3

8-element Vector{Float64}:
  23.271025228242667
   0.0
   0.0
  -2.4398388002589604e-5
  99.9999999993496
  19.595646098658175
   0.002083620500716367
 -16.683358148175163

In [23]:
gen_1

14-element Vector{Float64}:
 135.72894330789333
   2.726015543361044e-11
   0.0
   0.0
   0.0
   0.0
   0.0
   0.0
   0.0
   0.0
   0.0
   0.0
   0.0
   0.0

In [25]:
gen_2

30-element Vector{Float64}:
 45.71106939960538
 79.99999999995171
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
 39.99999999978272
  ⋮
  0.0
  0.0
  0.0
 23.488930600487254
  1.4485075152381827e-10
  0.0
  0.0
  0.0
  2.803562930263671e-11
  0.0
  0.0
  0.0

In [26]:
gen_3

5-element Vector{Float64}:
 1.595472312812994e-11
 6.820997093669916e-12
 0.0
 0.0
 0.0

In [30]:
show(stdout, "text/plain", gen_2)

30-element Vector{Float64}:
 45.71106939960538
 79.99999999995171
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
 39.99999999978272
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
 23.488930600487254
  1.4485075152381827e-10
  0.0
  0.0
  0.0
  2.803562930263671e-11
  0.0
  0.0
  0.0